In [1]:
import os
%load_ext autoreload
%autoreload 2
import pandas as pd
import pyspark
import pyarrow

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
os.environ['SPARK_HOME'] = 'C:/Users/saul2/Spark_DF/spark-3.5.5-bin-hadoop3'
os.environ['HADOOP_HOME'] = 'C:/Users/saul2/Spark_DF/spark-3.5.5-bin-hadoop3'
os.environ['JAVA_HOME'] = 'C:/Program Files/Java/jdk1.8.0_202'

In [5]:
#spark = SparkSession.builder.appName("Practice").\
#    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
#    .config("spark.hadoop.hadoop.native.io", "false").getOrCreate()
spark = SparkSession.builder.appName("Practice").getOrCreate()

In [6]:
csv_file = 'Students_Grading_Dataset.csv'
df = spark.read.option("header", True).csv(csv_file)
df.createOrReplaceTempView("Students")

In [21]:
# Show Column Names
query = 'DESC STUDENTS'
result = spark.sql(query)
result.show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|          Student_ID|   string|   NULL|
|          First_Name|   string|   NULL|
|           Last_Name|   string|   NULL|
|               Email|   string|   NULL|
|              Gender|   string|   NULL|
|                 Age|   string|   NULL|
|          Department|   string|   NULL|
|      Attendance (%)|   string|   NULL|
|       Midterm_Score|   string|   NULL|
|         Final_Score|   string|   NULL|
|     Assignments_Avg|   string|   NULL|
|         Quizzes_Avg|   string|   NULL|
| Participation_Score|   string|   NULL|
|      Projects_Score|   string|   NULL|
|         Total_Score|   string|   NULL|
|               Grade|   string|   NULL|
|Study_Hours_per_Week|   string|   NULL|
|Extracurricular_A...|   string|   NULL|
|Internet_Access_a...|   string|   NULL|
|Parent_Education_...|   string|   NULL|
+--------------------+---------+-------+
only showing top

In [8]:
# Students Older than 21
query = "SELECT * FROM STUDENTS WHERE AGE > 21"
result = spark.sql(query)
#result.show()

In [23]:
# Students with a total_score higher than their department_avg
query = '''
    WITH DEPT_AVG AS (
        SELECT DEPARTMENT, ROUND(AVG(TOTAL_SCORE),2) AVG_SCORE
        FROM STUDENTS 
        GROUP BY DEPARTMENT
    )
    SELECT STUDENTS.DEPARTMENT, STUDENT_ID, FIRST_NAME, LAST_NAME, TOTAL_SCORE, DEPT_AVG.AVG_SCORE
    FROM STUDENTS, DEPT_AVG
    WHERE STUDENTS.DEPARTMENT = DEPT_AVG.DEPARTMENT
        AND STUDENTS.TOTAL_SCORE > DEPT_AVG.AVG_SCORE
    ORDER BY STUDENTS.DEPARTMENT, STUDENT_ID
    '''
result = spark.sql(query)
#result.show()

In [47]:
# Find the top 5 students with the highest final exam scores.
query = '''
        WITH TOP_5 AS (
            SELECT STUDENT_ID, FIRST_NAME, LAST_NAME, FINAL_SCORE, RANK()OVER(ORDER BY FINAL_SCORE DESC) RANK
            FROM STUDENTS
            ORDER BY FINAL_SCORE DESC
        )
        SELECT STUDENT_ID, FIRST_NAME, LAST_NAME, FINAL_SCORE
        FROM TOP_5
        WHERE RANK <= 5
    '''
result = spark.sql(query)
result.show()

+----------+----------+---------+-----------+
|STUDENT_ID|FIRST_NAME|LAST_NAME|FINAL_SCORE|
+----------+----------+---------+-----------+
|     S3688|     Ahmed|    Brown|      99.98|
|     S5152|      John|    Smith|      99.98|
|     S1957|       Ali| Williams|      99.97|
|     S4304|      Sara|    Davis|      99.97|
|     S1977|      Liam|    Brown|      99.96|
|     S4522|     Maria| Williams|      99.96|
+----------+----------+---------+-----------+



In [19]:
query = '''
    SELECT COUNT(*)
    FROM STUDENTS 
    '''
result = spark.sql(query)
result.show()

+--------+
|count(1)|
+--------+
|    5000|
+--------+



In [ ]:
# Make a directory for my files
output_dir = 'results'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Conver my result to a Pandas Dataframe. PySpark is not working well to conver it into a parquet/csv file.
#pandas_df = result.toPandas()
#path = os.path.join(output_dir, 'student_results.parquet')
#pandas_df.to_parquet(path, engine='pyarrow') 
#path = os.path.join(output_dir, 'student_results.csv')
#pandas_df.to_csv(path, index=False)